In [1]:
# DSC540
# Weeks 9 & 10
# Milestone 4
# Author: Nathanael Ochoa
# 05/19/2024

# Connecting to an API/Pulling in the Data and Cleaning/Formatting

None of the following data is my own and is used for educational purposes. The data is collected from OpenWeather's [Current Weather Data API ](https://openweathermap.org/current). I mush also use OpenWeather's [Geocoding API](https://openweathermap.org/api/geocoding-api) to retrieve Seattle's geographical coordinates, which are required to use the Current Weather Data API. 

In [2]:
# import packages
import pandas as pd
import datetime
import requests
import json

In [3]:
# Retrieve api key from json file
with open("APIkeys.json") as f:
    keys = json.load(f)
    ow_api = keys["OpenWeather"]

In [4]:
def getCoordinates():
    # My analysis is based in Seattle, WA so these variables are set
    city_name = "Seattle"
    state_code = "WA"
    country_code = "US"
    
    # Beginning url
    beg = "http://api.openweathermap.org/geo/1.0/direct?q="
    
    # Combine beg, codes, and api key
    geo_url = beg + f"{city_name},{state_code},{country_code}&appid={ow_api}"
    
    # Now call the API and store the coordinates into a list
    # API call
    response = requests.get(geo_url)
    if response.status_code == 200:
        geo_data = response.json()
    else:
        print(f"Data retrieval failted. Status code: {response.status_code}")
        
    # Create new list with 'lat' and 'lon' values
    geo_list = [{"lat": geo_data[0]["lat"], "lon": geo_data[0]["lon"]}]
    
    return geo_list

In [5]:
def getData(): 
    # Beginning url
    beg = "https://api.openweathermap.org/data/2.5/weather?"
    
    # Call getCoordinates to get the 'lat' and 'lon' values
    coord = getCoordinates()
    
    # Latitude
    lat = f"lat={coord[0]['lat']}"
    
    # Longitude
    lon = f"&lon={coord[0]['lon']}"
    
    # Key and units, temperature units will be in Fahrenheit
    end = f"&appid={ow_api}&units=imperial"
    
    # Combine
    url = beg + lat + lon + end
    
    # API call
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Data retrieval failted. Status code: {response.status_code}")
        
    return data

In [6]:
# Function call that gets API data
weather_json = getData()

In [7]:
# View json data
weather_json

{'coord': {'lon': -122.3301, 'lat': 47.6038},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 62.98,
  'feels_like': 62.08,
  'temp_min': 59.43,
  'temp_max': 67.32,
  'pressure': 1021,
  'humidity': 66},
 'visibility': 10000,
 'wind': {'speed': 10.36, 'deg': 220},
 'clouds': {'all': 40},
 'dt': 1716838010,
 'sys': {'type': 2,
  'id': 2041694,
  'country': 'US',
  'sunrise': 1716812341,
  'sunset': 1716868466},
 'timezone': -25200,
 'id': 5809844,
 'name': 'Seattle',
 'cod': 200}

In [8]:
# Convert any lists in the JSON data to dictionaries
# This will help make sure every item will have its own data frame column
for key in weather_json:
    if isinstance(weather_json[key], type([])):
        weather_json[key] = weather_json[key][0]
    else:
        pass

In [9]:
# View changes
weather_json

{'coord': {'lon': -122.3301, 'lat': 47.6038},
 'weather': {'id': 802,
  'main': 'Clouds',
  'description': 'scattered clouds',
  'icon': '03d'},
 'base': 'stations',
 'main': {'temp': 62.98,
  'feels_like': 62.08,
  'temp_min': 59.43,
  'temp_max': 67.32,
  'pressure': 1021,
  'humidity': 66},
 'visibility': 10000,
 'wind': {'speed': 10.36, 'deg': 220},
 'clouds': {'all': 40},
 'dt': 1716838010,
 'sys': {'type': 2,
  'id': 2041694,
  'country': 'US',
  'sunrise': 1716812341,
  'sunset': 1716868466},
 'timezone': -25200,
 'id': 5809844,
 'name': 'Seattle',
 'cod': 200}

In [10]:
# Convert to a data frame
api_data = pd.json_normalize(weather_json)

In [11]:
# View data frame
api_data

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,weather.id,...,main.pressure,main.humidity,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,stations,10000,1716838010,-25200,5809844,Seattle,200,-122.3301,47.6038,802,...,1021,66,10.36,220,40,2,2041694,US,1716812341,1716868466


In [12]:
# Better view of the data frame
api_data.stack()

0  base                           stations
   visibility                        10000
   dt                           1716838010
   timezone                         -25200
   id                              5809844
   name                            Seattle
   cod                                 200
   coord.lon                     -122.3301
   coord.lat                       47.6038
   weather.id                          802
   weather.main                     Clouds
   weather.description    scattered clouds
   weather.icon                        03d
   main.temp                         62.98
   main.feels_like                   62.08
   main.temp_min                     59.43
   main.temp_max                     67.32
   main.pressure                      1021
   main.humidity                        66
   wind.speed                        10.36
   wind.deg                            220
   clouds.all                           40
   sys.type                              2
   sys.id  

## Step 1 - Drop unnecessary columns

Certain values/columns are internal parameters so they will be dropped. Others are pointless for this analysis so they will also be dropped.

In [13]:
# Drop columns
api_data.drop(["base", "visibility", "timezone", "id", "cod", "weather.id", "weather.icon", "main.pressure", "main.humidity", 
              "wind.deg", "sys.type", "sys.id", "sys.country", "sys.sunrise", "sys.sunset"], axis = 1, inplace = True)

In [14]:
# View changes
api_data.stack()

0  dt                           1716838010
   name                            Seattle
   coord.lon                     -122.3301
   coord.lat                       47.6038
   weather.main                     Clouds
   weather.description    scattered clouds
   main.temp                         62.98
   main.feels_like                   62.08
   main.temp_min                     59.43
   main.temp_max                     67.32
   wind.speed                        10.36
   clouds.all                           40
dtype: object

## Step 2 - Split the 'dt' column into separate columns

The API documentation states that the **dt** response field is the 'time of data calculation, unix, UTC.' I can get both the time and date from this column. I will create the following columns: **retrieval_date**, **retrieval_time**, **r_date_month**, **r_date_day**, and **r_date_year**.

In [15]:
# Convert the column
timestamp = datetime.datetime.fromtimestamp(api_data["dt"].iloc[0])

In [16]:
# Create new columns
api_data["retrieval_date"] = timestamp.strftime("%m-%d-%Y")
api_data["retrieval_time"] = timestamp.strftime("%H:%M:%S")
api_data["r_date_month"] = timestamp.strftime("%m")
api_data["r_date_day"] = timestamp.strftime("%d")
api_data["r_date_year"] = timestamp.strftime("%Y")

In [17]:
# Drop 'dt' column
api_data.drop(["dt"], axis = 1, inplace = True)

In [18]:
# View changes
api_data.stack()

0  name                            Seattle
   coord.lon                     -122.3301
   coord.lat                       47.6038
   weather.main                     Clouds
   weather.description    scattered clouds
   main.temp                         62.98
   main.feels_like                   62.08
   main.temp_min                     59.43
   main.temp_max                     67.32
   wind.speed                        10.36
   clouds.all                           40
   retrieval_date               05-27-2024
   retrieval_time                 12:26:50
   r_date_month                         05
   r_date_day                           27
   r_date_year                        2024
dtype: object

## Step 3 - Add a 'state' column

I will add a state column using Washington's state code, WA.

In [19]:
api_data["state"] = "WA"

In [20]:
# View changes
api_data.stack()

0  name                            Seattle
   coord.lon                     -122.3301
   coord.lat                       47.6038
   weather.main                     Clouds
   weather.description    scattered clouds
   main.temp                         62.98
   main.feels_like                   62.08
   main.temp_min                     59.43
   main.temp_max                     67.32
   wind.speed                        10.36
   clouds.all                           40
   retrieval_date               05-27-2024
   retrieval_time                 12:26:50
   r_date_month                         05
   r_date_day                           27
   r_date_year                        2024
   state                                WA
dtype: object

## Step 4 - Change column headers

Plenty of the column headers are not simple and easy to read, so I will be changing that. For example, I will change **weather.main** to just **weather**.

In [21]:
# Rename columns
api_data.rename(columns = {"name": "city", "coord.lon": "longitude", "coord.lat": "latitude", "weather.main": "current_weather", 
                          "weather.description": "w_description", "main.temp": "temperature", 
                           "main.feels_like": "temp_feels_like", "main.temp_min": "min_temp", "main.temp_max": "max_temp",
                          "wind.speed": "wind_speed_mph", "clouds.all": "cloudiness_%"}, inplace = True)

In [22]:
# View changes
api_data.stack()

0  city                        Seattle
   longitude                 -122.3301
   latitude                    47.6038
   current_weather              Clouds
   w_description      scattered clouds
   temperature                   62.98
   temp_feels_like               62.08
   min_temp                      59.43
   max_temp                      67.32
   wind_speed_mph                10.36
   cloudiness_%                     40
   retrieval_date           05-27-2024
   retrieval_time             12:26:50
   r_date_month                     05
   r_date_day                       27
   r_date_year                    2024
   state                            WA
dtype: object

## Step 5 - Rearrange column order

Everything is looking good so far but the column order is odd, so I will change that.

In [23]:
api_data = api_data[["city", "state", "latitude", "longitude", "current_weather", "w_description", "temperature", 
                     "temp_feels_like", "min_temp", "max_temp", "wind_speed_mph", "cloudiness_%", "retrieval_date", 
                     "retrieval_time", "r_date_month", "r_date_day", "r_date_year"]]

In [24]:
# View changes
api_data.stack()

0  city                        Seattle
   state                            WA
   latitude                    47.6038
   longitude                 -122.3301
   current_weather              Clouds
   w_description      scattered clouds
   temperature                   62.98
   temp_feels_like               62.08
   min_temp                      59.43
   max_temp                      67.32
   wind_speed_mph                10.36
   cloudiness_%                     40
   retrieval_date           05-27-2024
   retrieval_time             12:26:50
   r_date_month                     05
   r_date_day                       27
   r_date_year                    2024
dtype: object

## Step 6 - Check column data types

The last thing left to do is check the column data types. There may be some errors or there may be a need to change a column's default type.

In [25]:
# View column data types
api_data.dtypes

city                object
state               object
latitude           float64
longitude          float64
current_weather     object
w_description       object
temperature        float64
temp_feels_like    float64
min_temp           float64
max_temp           float64
wind_speed_mph     float64
cloudiness_%         int64
retrieval_date      object
retrieval_time      object
r_date_month        object
r_date_day          object
r_date_year         object
dtype: object

Everything looks good, so I will leave everything as is. I left the date columns as object/str data types in a previous milestone, so I be doing the same this time around as well.

## View final API data:

In [26]:
# View as data frame
api_data

,city,state,latitude,longitude,current_weather,w_description,temperature,temp_feels_like,min_temp,max_temp,wind_speed_mph,cloudiness_%,retrieval_date,retrieval_time,r_date_month,r_date_day,r_date_year
0,Seattle,WA,47.6038,-122.3301,Clouds,scattered clouds,62.98,62.08,59.43,67.32,10.36,40,05-27-2024,12:26:50,05,27,2024


In [27]:
# Better view
api_data.stack()

0  city                        Seattle
   state                            WA
   latitude                    47.6038
   longitude                 -122.3301
   current_weather              Clouds
   w_description      scattered clouds
   temperature                   62.98
   temp_feels_like               62.08
   min_temp                      59.43
   max_temp                      67.32
   wind_speed_mph                10.36
   cloudiness_%                     40
   retrieval_date           05-27-2024
   retrieval_time             12:26:50
   r_date_month                     05
   r_date_day                       27
   r_date_year                    2024
dtype: object

I dropped any unnecessary columns and converted the **dt** column into date and time. Other than that, I made no changes to draw any ethical concerns. I also changed the column headers to be 'human-readable'. Once all transformations were completed, I rearranged the columns to ensure the data frame was also 'human-readable'. There were no other changes made, and I made no assumptions while cleaning up the JSON data.

In [28]:
# Download as CSV to use in Milestone 5
api_data.to_csv("M5_api_data.csv")